In [3]:
%matplotlib inline

import datetime
import matplotlib.pyplot as plt
import matplotlib
import xarray as xr
import numpy as np
import pandas as pd
import s3fs
import fsspec
import dask
from dask.distributed import performance_report, Client, progress


In [4]:
client = Client()
client

2022-10-10 20:41:06,511 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/b_/gxd85x_n4jv7sxtvcc1sv46c0000gp/T/dask-worker-space/worker-_tx0j227', purging
2022-10-10 20:41:06,528 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/b_/gxd85x_n4jv7sxtvcc1sv46c0000gp/T/dask-worker-space/worker-dsvm15sc', purging
2022-10-10 20:41:06,528 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/b_/gxd85x_n4jv7sxtvcc1sv46c0000gp/T/dask-worker-space/worker-xn1fonf1', purging
2022-10-10 20:41:06,528 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/b_/gxd85x_n4jv7sxtvcc1sv46c0000gp/T/dask-worker-space/worker-15mxqd0v', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 8.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:59989,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:60071,Total threads: 2
Dashboard: http://127.0.0.1:60074/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:59996,


In [ ]:
# sstk 34

In [5]:
base_url = 'https://rda.ucar.edu/thredds/dodsC/files/g/ds633.1_nc/e5.moda.an.sfc/'
base_url2 = '/e5.moda.an.sfc.128_034_sstk.ll025sc.'
base_url3 = '.nc'

# period of interest
pr = pd.date_range(start='1979-01',end='2021-12', freq='AS')

file_list=[]
for dt in pr:
    # get recent year and month
    year = dt.strftime('%Y')
    month = dt.strftime('%Y%m%d%H')
    month2 = (dt + pd.DateOffset(months=11)).strftime('%Y%m%d%H')

    # build complete file name
    single_file=(base_url+year+base_url2+month+'_'+month2+base_url3)
    
    file_list.append(single_file)


In [6]:
file_list

['https://rda.ucar.edu/thredds/dodsC/files/g/ds633.1_nc/e5.moda.an.sfc/1979/e5.moda.an.sfc.128_034_sstk.ll025sc.1979010100_1979120100.nc',
 'https://rda.ucar.edu/thredds/dodsC/files/g/ds633.1_nc/e5.moda.an.sfc/1980/e5.moda.an.sfc.128_034_sstk.ll025sc.1980010100_1980120100.nc',
 'https://rda.ucar.edu/thredds/dodsC/files/g/ds633.1_nc/e5.moda.an.sfc/1981/e5.moda.an.sfc.128_034_sstk.ll025sc.1981010100_1981120100.nc',
 'https://rda.ucar.edu/thredds/dodsC/files/g/ds633.1_nc/e5.moda.an.sfc/1982/e5.moda.an.sfc.128_034_sstk.ll025sc.1982010100_1982120100.nc',
 'https://rda.ucar.edu/thredds/dodsC/files/g/ds633.1_nc/e5.moda.an.sfc/1983/e5.moda.an.sfc.128_034_sstk.ll025sc.1983010100_1983120100.nc',
 'https://rda.ucar.edu/thredds/dodsC/files/g/ds633.1_nc/e5.moda.an.sfc/1984/e5.moda.an.sfc.128_034_sstk.ll025sc.1984010100_1984120100.nc',
 'https://rda.ucar.edu/thredds/dodsC/files/g/ds633.1_nc/e5.moda.an.sfc/1985/e5.moda.an.sfc.128_034_sstk.ll025sc.1985010100_1985120100.nc',
 'https://rda.ucar.edu/thre

In [7]:
ds = xr.open_mfdataset(file_list, chunks={'time': '100MB'}).sel(latitude=slice(65, -65, 1),longitude=slice(120, 300, 1)).compute()

In [8]:
ds

<xarray.Dataset>
Dimensions:    (latitude: 521, longitude: 721, time: 516)
Coordinates:
  * latitude   (latitude) float64 65.0 64.75 64.5 64.25 ... -64.5 -64.75 -65.0
  * longitude  (longitude) float64 120.0 120.2 120.5 120.8 ... 299.5 299.8 300.0
  * time       (time) datetime64[ns] 1979-01-01 1979-02-01 ... 2021-12-01
Data variables:
    utc_date   (time) int32 1979010100 1979020100 ... 2021110100 2021120100
    SSTK       (time, latitude, longitude) float32 nan nan nan ... nan nan nan
Attributes:
    DATA_SOURCE:                     ECMWF: https://cds.climate.copernicus.eu...
    NETCDF_CONVERSION:               CISL RDA: Conversion from ECMWF GRIB 1 d...
    NETCDF_VERSION:                  4.6.1
    CONVERSION_PLATFORM:             Linux casper02 3.10.0-693.21.1.el7.x86_6...
    CONVERSION_DATE:                 Mon Nov 11 07:46:57 MST 2019
    Conventions:                     CF-1.6
    NETCDF_COMPRESSION:              NCO: Precision-preserving compression to...
    history:                         Mon Nov 11 07:46:58 2019: ncks -4 --ppc ...
    NCO:                             netCDF Operators version 4.7.9 (Homepage...
    DODS_EXTRA.Unlimited_Dimension:  time

In [9]:
gb = ds.SSTK.groupby('time.month')
gb

DataArrayGroupBy, grouped over 'month'
12 groups with labels 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12.

In [10]:
anom = gb - gb.mean(dim='time')

In [11]:
anom

<xarray.DataArray 'SSTK' (time: 516, latitude: 521, longitude: 721)>
array([[[            nan,             nan,             nan, ...,
         -8.17871094e-03, -1.33056641e-02, -1.83410645e-02],
        [            nan,             nan,             nan, ...,
         -1.33972168e-02, -2.06604004e-02, -2.74963379e-02],
        [            nan,             nan,             nan, ...,
         -1.77917480e-02, -2.50244141e-02, -3.21960449e-02],
        ...,
        [-3.00506592e-01, -3.01300049e-01, -3.02459717e-01, ...,
                     nan,             nan,             nan],
        [-8.54736328e-01, -8.40728760e-01, -8.27301025e-01, ...,
                     nan,             nan,             nan],
        [-8.32458496e-01, -8.32183838e-01, -8.46466064e-01, ...,
                     nan,             nan,             nan]],

       [[            nan,             nan,             nan, ...,
         -5.18798828e-04, -7.01904297e-04, -8.85009766e-04],
        [            nan,             nan,             nan, ...,
         -7.01904297e-04, -1.92260742e-03, -3.54003906e-03],
        [            nan,             nan,             nan, ...,
         -1.58691406e-03, -4.24194336e-03, -6.86645508e-03],
...
                     nan,             nan,             nan],
        [-6.92749023e-03, -8.08715820e-03, -9.15527344e-03, ...,
                     nan,             nan,             nan],
        [-8.54492188e-04, -1.40380859e-03, -1.40380859e-03, ...,
                     nan,             nan,             nan]],

       [[            nan,             nan,             nan, ...,
          9.37194824e-02,  6.34460449e-02,  3.24096680e-02],
        [            nan,             nan,             nan, ...,
          7.76062012e-02,  5.41687012e-02,  3.11279297e-02],
        [            nan,             nan,             nan, ...,
          7.62939453e-02,  5.80749512e-02,  4.01306152e-02],
        ...,
        [-1.10382080e-01, -8.47473145e-02, -6.00891113e-02, ...,
                     nan,             nan,             nan],
        [-3.03283691e-01, -2.89428711e-01, -2.75299072e-01, ...,
                     nan,             nan,             nan],
        [-1.43981934e-01, -1.37939453e-01, -1.40350342e-01, ...,
                     nan,             nan,             nan]]],
      dtype=float32)
Coordinates:
  * latitude   (latitude) float64 65.0 64.75 64.5 64.25 ... -64.5 -64.75 -65.0
  * longitude  (longitude) float64 120.0 120.2 120.5 120.8 ... 299.5 299.8 300.0
  * time       (time) datetime64[ns] 1979-01-01 1979-02-01 ... 2021-12-01
    month      (time) int64 1 2 3 4 5 6 7 8 9 10 11 ... 2 3 4 5 6 7 8 9 10 11 12

In [29]:
# Save dataset of sst
#ds.to_netcdf('SST.nc')

In [30]:
client.close()

2022-10-10 18:06:45,775 - distributed.nanny - WARNING - Worker process still alive after 3.999995040893555 seconds, killing
2022-10-10 18:06:45,849 - distributed.nanny - WARNING - Worker process still alive after 3.9999982833862306 seconds, killing
2022-10-10 18:06:45,858 - distributed.nanny - WARNING - Worker process still alive after 3.999999237060547 seconds, killing
2022-10-10 18:06:45,869 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
